In [ ]:
from datetime import datetime
import os

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
#import matplotlib.pyplot as plt
import MySQLdb as mdb
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

%matplotlib inline

In [ ]:
def create_connection():
    default_file = os.path.join(os.path.expanduser('~/'), '.atefd.cnf')
    return mdb.connect(read_default_file=default_file, db='EFD')

def get_engine():
    return create_engine('mysql+mysqldb://', creator=create_connection)

In [ ]:
engine = get_engine()

In [ ]:
d1 = datetime(2019, 4, 3)
d2 = datetime(2019, 4, 5)

In [ ]:
query_base = 'select {0} from {1} where {1}.date_time between \'{2}\' and \'{3}\';'

In [ ]:
atcamera_info = query_base.format('date_time,private_sndStamp,imageName,timestamp',
                                  'ATCamera_logevent_endReadout', d1, d2)

In [ ]:
atcamera_df = pd.read_sql_query(atcamera_info, engine)

In [ ]:
atarchiver_info = query_base.format('date_time,private_sndStamp,statusCode,description',
                                    'ATArchiver_logevent_processingStatus', d1, d2)

In [ ]:
atarchiver_df = pd.read_sql_query(atarchiver_info, engine)
#print(atarchiver_df)

In [ ]:
aths_info = query_base.format('date_time,private_sndStamp,url',
                              'ATHeaderService_logevent_largeFileObjectAvailable', d1, d2)
extra = ' and ATHeaderService_logevent_largeFileObjectAvailable.generator=\'ATHeaderService\''
parts = aths_info.split(';')
aths_info = parts[0] + extra + parts[1]
#print(aths_info)

In [ ]:
aths_df = pd.read_sql_query(aths_info, engine)

In [ ]:
efd_info = query_base.format('date_time,private_sndStamp,url',
                             'EFD_logevent_largeFileObjectAvailable', d1, d2)
extra = ' and EFD_logevent_largeFileObjectAvailable.generator=\'ATHeaderService\''
parts = efd_info.split(';')
efd_info = parts[0] + extra + parts[1]

In [ ]:
efd_df = pd.read_sql_query(efd_info, engine)

In [ ]:
atcamera_send_time = []
atarchiver_send_time = []
atarchiver_arrival_time = []
aths_send_time = []
efd_send_time = []
for row in atcamera_df.itertuples():
    atcamera_time = row.private_sndStamp
    atcamera_image = row.imageName
    index = atarchiver_df['description'].str.contains(f"{atcamera_image}", na=False)
    try:
        if atarchiver_df['statusCode'][index].values[0] == 100.0:
            atarchiver_time = atarchiver_df['private_sndStamp'][index].values[0]
            have_archiver_file = True
        else:
            have_archiver_file = False
    except IndexError:
        have_archiver_file = False
    index = aths_df['url'].str.contains(f"{atcamera_image}", na=False)
    try:
        aths_time = aths_df['private_sndStamp'][index].values[0]
        have_header_file = True
    except IndexError:
        have_header_file = False
    #print(atarchive_send_time.values)
    index = efd_df['url'].str.contains(f"{atcamera_image}", na=False)
    try:
        efd_time = efd_df['private_sndStamp'][index].values[0]
        have_efd_header_file = True
    except IndexError:
        have_efd_header_file = False
    
    if have_header_file and have_archiver_file and have_efd_header_file:
        atcamera_send_time.append(atcamera_time)
        atarchiver_send_time.append(atarchiver_time)
        aths_send_time.append(aths_time)
        efd_send_time.append(efd_time)
        atarchiver_arrival_time.append(atarchiver_time - atcamera_time)

In [ ]:
atcamera_send_time_array = np.array(atcamera_send_time)
atarchiver_send_time_array = np.array(atarchiver_send_time)
atarchiver_arrival_time_array = np.array(atarchiver_arrival_time)
aths_send_time_array = np.array(aths_send_time)
efd_send_time_array = np.array(efd_send_time)
time_diff_str = 'T^ATA_PS_Send - T^ATC_EOR_Send'

In [ ]:
hist = np.histogram(atarchiver_arrival_time_array, bins=np.arange(start=0, stop=100, step=2))
aat = hv.Histogram(hist, kdims=hv.Dimension('time differences', label=time_diff_str, unit='s'),
                   label='Distribution of 𝚫T')

In [ ]:
aat

In [ ]:
cst_vs_aat = hv.Scatter((atcamera_send_time_array.astype('datetime64[s]'), atarchiver_arrival_time_array),
                        kdims='T^ATC_EOR_Send', vdims=hv.Dimension('time difference', label=time_diff_str, unit='s'),
                        label='𝚫T During Image Taking')
cst_vs_aat.opts(padding=(0.05, 0.05))

In [ ]:
delta_t1 = (aths_send_time_array - atcamera_send_time_array)
min_x = np.min(delta_t1)
max_x = np.max(delta_t1)
hist = np.histogram(delta_t1, bins=np.linspace(min_x, max_x, 50))
ahst = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
                                             label='T^ATHS_LFOA_Send - T^ATC_EOR_Send', unit='s'),
                    label='Distribution of 𝚫T1')

In [ ]:
ahst

In [ ]:
delta_t2 = (efd_send_time_array - aths_send_time_array)
min_x = np.min(delta_t2)
max_x = np.max(delta_t2)
hist = np.histogram(delta_t2, bins=np.linspace(min_x, max_x, 50))
est = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
                                             label='T^EFD_LFOA_Send - T^ATHS_LFOA_Send', unit='s'),
                    label='Distribution of 𝚫T2')

In [ ]:
est

In [ ]:
delta_t3 = (efd_send_time_array - atcamera_send_time_array)
min_x = np.min(delta_t3)
max_x = np.max(delta_t3)
hist = np.histogram(delta_t3, bins=np.linspace(min_x, max_x, 50))
hfst = hv.Histogram(hist, kdims=hv.Dimension('time differences', 
                                             label='T^EFD_LFOA_Send - T^ATC_EOR_Send', unit='s'),
                    label='Distribution of 𝚫T3')

In [ ]:
hfst

In [ ]:
indexes = np.where(atarchiver_arrival_time_array > 40.0)
for c, a in zip(atarchiver_arrival_time_array[indexes], atcamera_send_time_array.astype('datetime64[s]')[indexes]):
    print(f"{c:.2f}\t{a}")